## Understanding and Predicting Property Maintenance Fines

This script is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)), and was created as part of an assessment for a Coursera course on Machine Learning. 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. Here, the task is to predict whether a given blight ticket will be paid on time.

All data is provided through the [Detroit Open Data Portal](https://data.detroitmi.gov/). 
___

There are two data files for use in training and validating models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** 

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Predictions are given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

The assessment criteria used is the AUC score computed for the classifier. A model which with an AUROC of 0.7 is deemed good; over 0.75 is deemed excellent.
___

I create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, I return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       

In [1]:
import pandas as pd
import numpy as np

In [29]:
def blight_model():
    
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import Ridge,Lasso
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import roc_auc_score
    from sklearn.svm import LinearSVC
    from pandas.api.types import is_string_dtype
    from pandas.api.types import is_numeric_dtype
    from sklearn.preprocessing import MinMaxScaler
    
    # load data
    df = pd.read_csv('train.csv',encoding='latin_1')

    # remove unwanted features
    drop_cols = ['payment_amount','payment_date','payment_status','balance_due',
                 'collection_status','compliance_detail','mailing_address_str_number']
    df.drop(drop_cols,axis=1,inplace=True)

    # remove Null compliance per instructions
    df = df[~np.isnan(df['compliance'])]
    
    # change non-numeric data into numeric using dict
    str_cols = ['agency_name','violation_code','disposition']
    numeric_df = df

    for col in str_cols:
        this_col = df[col]

        # convert to all lowercase
        numeric_df[col] = this_col.str.lower()

        uqs = pd.unique(numeric_df[col])
        col_dict = {uqs[i]:i for i in range(len(uqs))}

        numeric_df[col].replace(col_dict,inplace=True)

    # remove columns that have all string values
    str_cols = numeric_df.columns[[is_string_dtype(df[i]) for i in df.columns]]
    numeric_df.drop(str_cols,axis=1,inplace=True)

    # remove columns that have all null values
    null_cols = numeric_df.columns[numeric_df.isnull().all(axis=0)]
    numeric_df.drop(null_cols,axis=1,inplace=True)

    # remove rows that have any null values
    null_rows = numeric_df.isnull().any(axis=1)
    numeric_df = numeric_df[~null_rows]
    
    # separate out features and labels
    X = numeric_df.drop('compliance',axis=1)
    y = numeric_df['compliance']


    # split into test and training sets
    X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)

    # normalize
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)
    
    # create and fit model
    mdl = Ridge(alpha=0.2).fit(X_train_scaled,y_train)
    
    
    # Print ROC AUC score
    y_pred = mdl.predict(X_test_scaled)
    train_score = roc_auc_score(y_test,y_pred)
    print('Training score for the 2004-2011 dataset = {0:.2f}%'.format(100*train_score))
    
    
    # Test on test dataset
    df_test = pd.read_csv('test.csv',encoding='latin_1')

    # Get the features
    numeric_df_test = df_test

    # replace strings with integer values where appropriate
    str_cols = ['agency_name','violation_code','disposition']

    for col in str_cols:
        this_col = df_test[col]

        # convert to all lowercase
        numeric_df_test[col] = this_col.str.lower()

        uqs = pd.unique(numeric_df_test[col])
        col_dict = {uqs[i]:i for i in range(len(uqs))}

        numeric_df_test[col].replace(col_dict,inplace=True)


    # remove columns that still have all string values
    str_cols = numeric_df_test.columns[[is_string_dtype(df_test[i]) for i in df_test.columns]]
    numeric_df_test.drop(str_cols,axis=1,inplace=True)

    # remove columns that have all null values
    null_cols = numeric_df_test.columns[numeric_df_test.isnull().all(axis=0)]
    numeric_df_test.drop(null_cols,axis=1,inplace=True)

    X = numeric_df_test


    # scale X
    X_scaled = scaler.transform(X)

    # Predictions from X
    y_pred_test = mdl.predict(X_scaled)
    
    # return series
    probs = pd.Series(y_pred_test,index=numeric_df_test['ticket_id'])

    
    return probs



In [30]:
y = blight_model()

C:\Users\richa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Training score for the 2004-2011 dataset = 78.38%


In [31]:
print('Predicted values for the 2012-2016 dataset:\n')
print(y)

Predicted values for the 2012-2016 dataset:

ticket_id
284932    0.040979
285362    0.022654
285361    0.039103
285338    0.044099
285346    0.045657
            ...   
376496    0.039820
376497    0.039820
376499    0.070925
376500    0.070918
369851    0.052124
Length: 61001, dtype: float64
